In [1]:
# 計時器
import time

# 大畫家
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
%matplotlib inline

# 各種好用套件
import os, random, sys, keras, math
import numpy as np

# k~k~k~~~~Keras core sturcture
from keras.models import Sequential, Model
from keras.layers import Activation, Input, Dense, Lambda, Dropout, concatenate
from keras.initializers import Ones
# concatenate 就是 merge

# some functions
from keras import initializers
from keras.constraints import max_norm, non_neg, unit_norm
from keras import backend as K
from keras import metrics

Using TensorFlow backend.


# Syntx to Neural Network    
* Step-by-step syntax to Neural Network

In [2]:
model = Sequential()

![Alt text](img/model_0.png)

In [3]:
model.add(Dense(3, input_shape=(2,)))

![Alt text](img/model_0_2x3.png)

In [4]:
model.add(Dense(2, kernel_constraint=unit_norm))

![Alt text](img/model_0_2x3_3x2.png)

In [5]:
model.add(Dense(3))

![Alt text](img/model_0_2x3_3x2_2x3.png)

In [6]:
model.add(Activation('sigmoid'))

![Alt text](img/model_0_2x3_3x2_2x3_activation.png)

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 9         
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


# Re-write above NN using Model API

In [8]:
x = Input(shape=(2,))

f = Dense(3, activation='relu')
u = f(x)

g = Dense(2, activation='relu')
v = g(u)

h = Dense(3, activation='sigmoid')
y = h(v)

model = Model(x, y)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 9         
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


In [9]:
x = Input(shape=(2,))
u = Dense(3, activation='relu')(x)
v = Dense(2, activation='relu')(u)
y = Dense(3, activation='sigmoid')(v)

model = Model(x, y)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 9         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 9         
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


In [10]:
# ...... just for fun
x = Input(shape=(2,))
y = Dense(3, activation='sigmoid')(Dense(2, activation='relu')(x))

model = Model(x, y)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 6         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 9         
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


# Black Magic - Shared Layer

In [11]:
x = Input(shape=(2,))

f = Dense(3, activation='relu')
u = f(x)

g = Dense(2, activation='relu')
v = g(u)

h = Dense(3, activation='sigmoid')
y = h(v)

model = Model(x, y)

# Black Magic - Branch and Merge

In [12]:
x = Input(shape=(2,), name='x_is_input')

f = Dense(1, activation='relu', name='f')
u_1 = f(x)

g = Dense(2, activation='relu', name='g')
u_2 = g(x)

# merge layer
merged_v = concatenate([u_1, u_2], name='merged_layer')

h = Dense(1, activation='softmax', name='h')
y = h(merged_v)

# modeling
model = Model(x, y)

In [13]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
x_is_input (InputLayer)          (None, 2)             0                                            
____________________________________________________________________________________________________
f (Dense)                        (None, 1)             3           x_is_input[0][0]                 
____________________________________________________________________________________________________
g (Dense)                        (None, 2)             6           x_is_input[0][0]                 
____________________________________________________________________________________________________
merged_layer (Concatenate)       (None, 3)             0           f[0][0]                          
                                                                   g[0][0]                 

# Black Magic - Branch and Merge

In [14]:
x = Input(shape=(2,), name='x_is_input')

f = Dense(3, activation='relu', name='before_branch')
u = f(x)

# red route
g_red = Dense(2, activation='relu',name='red')
v_1 = g_red(u)

# blue route
g_blue_1 = Dense(1, activation='sigmoid',name='blue_1')
v_blue = g_blue_1(u)

g_blue_2 = Dense(2, activation='sigmoid',name='blue_2')
v_2 = g_blue_2(v_blue)

# merge layer
merged_v = concatenate([v_1, v_2], name='merged_vector')

h = Dense(3, activation='softmax', name='before_output')
y = h(merged_v)

# modeling
model = Model(x, y)

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
x_is_input (InputLayer)          (None, 2)             0                                            
____________________________________________________________________________________________________
before_branch (Dense)            (None, 3)             9           x_is_input[0][0]                 
____________________________________________________________________________________________________
blue_1 (Dense)                   (None, 1)             4           before_branch[0][0]              
____________________________________________________________________________________________________
red (Dense)                      (None, 2)             8           before_branch[0][0]              
___________________________________________________________________________________________

# Black Magic - Customized Layer    
* For more detai, please read: https://keras.io/layers/core/#lambda

In [16]:
def my_avergae(args):
    return K.mean(args)

In [17]:
x = Input(shape=(3,), name='x_is_input')

f = Lambda(my_avergae, output_shape=(1,))
y = f(x)

model = Model(x, y)

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x_is_input (InputLayer)      (None, 3)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.predict(np.array([[4,2,3]]))

array([ 3.], dtype=float32)

# Black Magic - Customized Loss Function

In [20]:
x = Input(shape=(2,))

f = Dense(3, activation='relu')
u = f(x)

g = Dense(2, activation='relu')
v = g(u)

h = Dense(3, activation='sigmoid')
y = h(v)

model = Model(x, y)

In [21]:
# re-defined MSE by user
def my_loss(y_true, y_pred):
    error = y_true - y_pred
    square_error = K.square(error)
    mean_square_error = K.mean(square_error)
    return mean_square_error

def my_loss_2(y_true, y_pred):
    return K.mean(K.square(y_true - y_pred))

In [22]:
model.compile(loss=my_loss, optimizer='SGD')

# Black Magic Revisit – Submodel

In [23]:
x = Input(shape=(28*28,), name='x_is_input')

enc_1 = Dense(100, activation='relu', kernel_initializer=Ones(), name='x_to_encoder')
x_enc = enc_1(x)

enc_2 = Dense(2, activation='relu', kernel_initializer=Ones(),name='encoder_to_latent')
latent = enc_2(x_enc)

dec_2 = Dense(100, activation='relu', kernel_initializer=Ones(),name='latent_to_decoder')
x_dec = dec_2(latent)

dec_1 = Dense(28*28, activation='relu', kernel_initializer=Ones(),name='deconder_to_x')
x_reconstruct = dec_1(x_dec)

# modeling
autoencoder = Model(x, x_reconstruct)
autoencoder.compile(loss='mse', optimizer='SGD')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x_is_input (InputLayer)      (None, 784)               0         
_________________________________________________________________
x_to_encoder (Dense)         (None, 100)               78500     
_________________________________________________________________
encoder_to_latent (Dense)    (None, 2)                 202       
_________________________________________________________________
latent_to_decoder (Dense)    (None, 100)               300       
_________________________________________________________________
deconder_to_x (Dense)        (None, 784)               79184     
Total params: 158,186
Trainable params: 158,186
Non-trainable params: 0
_________________________________________________________________


In [24]:
encoder = Model(x, latent)
encoder.summary()

decoder_input = Input(shape=(2,), name='latent_as_input')

# dec_2 = Dense(100, activation='relu', name='latent_to_decoder')
# x_dec = dec_2(latent)
_dec = dec_2(decoder_input)

# dec_1 = Dense(28*28, activation='relu', name='deconder_to_x')
# x_reconstruct = dec_1(x_dec)
_reconstruct = dec_1(_dec)


decoder = Model(decoder_input, _reconstruct)
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x_is_input (InputLayer)      (None, 784)               0         
_________________________________________________________________
x_to_encoder (Dense)         (None, 100)               78500     
_________________________________________________________________
encoder_to_latent (Dense)    (None, 2)                 202       
Total params: 78,702
Trainable params: 78,702
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
latent_as_input (InputLayer) (None, 2)                 0         
_________________________________________________________________
latent_to_decoder (Dense)    (None, 100)               300       
_________________________________________________________________
decond

In [25]:
# single 28-by-28 matrix with all entry equals to 1
x_test = np.array([np.ones(28*28)])

In [26]:
# input x_test to see its reconstructed result
ae_output = autoencoder.predict(x_test)

# encode x_test to get encoded result
# then decode it to see the reconstruct result
x_test_encoded = encoder.predict(x_test)
enc_dec_output = decoder.predict(x_test_encoded)

# check if two results are the same
(ae_output == enc_dec_output).sum()

784

# Assignment Revisit - Variational Autoencoder    
* Understand how VAE works
* Define what we need to build VAE
* Review the code of VAE!